In [1]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [2]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [3]:
# define Tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# define Agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    description="一个通过使用工具为用户提供天气信息的智能体。",
    system_message="你是一个乐于助人的AI智能助手。擅长使用工具解决任务。任务完成后，回复<stop autogenworkshop>",
    tools=[get_weather],
)


In [4]:
# Define a termination condition. End the conversation if a specific text is mentioned.
termination = TextMentionTermination("<stop autogenworkshop>")

# Define Team with type set to RoundRobinGroupChat
agent_team = RoundRobinGroupChat(participants=[weather_agent], termination_condition=termination, max_turns=None)


In [5]:
 # 1、run() method to run the team
result = await agent_team.run(task="how about the weather of shanghai?")
print(result)

Error processing publish message for weather_agent/8875d724-22e8-4002-a7d2-b7bf0c28bcee
Traceback (most recent call last):
  File "/home/azureuser/azure-ai-agent-workshop/.venv/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 409, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/azureuser/azure-ai-agent-workshop/.venv/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/azureuser/azure-ai-agent-workshop/.venv/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 48, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/azureuser/azure-ai-agent-workshop/.venv/lib/python3.12/site-packages/autogen_core/_routed_agent.py", line 48

CancelledError: 

In [ ]:
# 2、run_stream()metho to run team
async for message in agent_team.run_stream(task="上海的天气如何?"):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='上海的天气如何?' type='TextMessage'


source='weather_agent' models_usage=RequestUsage(prompt_tokens=391, completion_tokens=17) content='上海的天气目前是73华氏度，阳光明媚。' type='TextMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=414, completion_tokens=14) content=[FunctionCall(id='call_MRVsoT3ae10yipnbtYWluv8n', arguments='{"city":"上海"}', name='get_weather')] type='ToolCallRequestEvent'
source='weather_agent' models_usage=None content=[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_MRVsoT3ae10yipnbtYWluv8n')] type='ToolCallExecutionEvent'
source='weather_agent' models_usage=None content='The weather in 上海 is 73 degrees and Sunny.' type='ToolCallSummaryMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=450, completion_tokens=17) content='上海的天气目前是73华氏度，阳光明媚。' type='TextMessage'
source='weather_agent' models_usage=RequestUsage(prompt_tokens=473, completion_tokens=17) content='上海的天气目前是73华氏度，阳光明媚。' type='TextMessage'
source='weather_agent' models_usage

In [ ]:
# Method runs the team and uses the official Console tool to output in the proper format
# 3、run_stream()
stream = agent_team.run_stream(task="上海的天气如何?")
await Console(stream)

---------- user ----------
上海的天气如何?
---------- weather_agent ----------
[FunctionCall(id='call_f2rANfvvq16VPEHdVTPHPYoP', arguments='{"city":"上海"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_f2rANfvvq16VPEHdVTPHPYoP')]
---------- weather_agent ----------
The weather in 上海 is 73 degrees and Sunny.


---------- weather_agent ----------
上海的天气是73°F（约23°C），阳光明媚。<stop autogenworkshop>


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海的天气如何?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=234, completion_tokens=14), content=[FunctionCall(id='call_f2rANfvvq16VPEHdVTPHPYoP', arguments='{"city":"上海"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in 上海 is 73 degrees and Sunny.', call_id='call_f2rANfvvq16VPEHdVTPHPYoP')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='weather_agent', models_usage=None, content='The weather in 上海 is 73 degrees and Sunny.', type='ToolCallSummaryMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=270, completion_tokens=25), content='上海的天气是73°F（约23°C），阳光明媚。<stop autogenworkshop>', type='TextMessage')], stop_reason="Text '<stop autogenworkshop>' mentioned")